In [1]:
import findspark
findspark.init()
findspark.find()

'C:\\Spark'

In [2]:
import os
import sys
os.environ['PYSPARK_PYTHON'] = sys.executable
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable

In [3]:
import pyspark
from pyspark import SparkContext
import datetime
import datetime
from pyspark.sql.functions import year, month, dayofmonth, expr,col,initcap, concat,lit,lower
from pyspark.sql.types import DateType, IntegerType
from pyspark.sql import SparkSession #Importing the Libraries
# Creating Spark Session
spark = SparkSession.builder.appName('SparkByExamples.com').getOrCreate()

In [4]:
df_cust = spark.read.format('org.apache.spark.sql.json').load("cdw_sapp_custmer.json")
df_cust.show()

+------+----------------+------------+-------------+--------------------+----------+----------+--------+----------+---------+--------------------+-----------+---------+-----------------+
|APT_NO|  CREDIT_CARD_NO|   CUST_CITY| CUST_COUNTRY|          CUST_EMAIL|CUST_PHONE|CUST_STATE|CUST_ZIP|FIRST_NAME|LAST_NAME|        LAST_UPDATED|MIDDLE_NAME|      SSN|      STREET_NAME|
+------+----------------+------------+-------------+--------------------+----------+----------+--------+----------+---------+--------------------+-----------+---------+-----------------+
|   656|4210653310061055|     Natchez|United States| AHooper@example.com|   1237818|        MS|   39120|      Alec|   Hooper|2018-04-21T12:49:...|         Wm|123456100|Main Street North|
|   829|4210653310102868|Wethersfield|United States| EHolman@example.com|   1238933|        CT|   06109|      Etta|   Holman|2018-04-21T12:49:...|    Brendan|123453023|    Redwood Drive|
|   683|4210653310116272|     Huntley|United States| WDunham@exam

In [5]:
#df_cust=df_cust.select("*", initcap(col('FIRST_NAME')))
#df_cust=df_cust.select("*", initcap(col('LAST_NAME'))).show()


df_cust = df_cust.withColumn("FIRST_NAME", initcap(col("FIRST_NAME")))
df_cust = df_cust.withColumn("MIDDLE_NAME", lower(col("MIDDLE_NAME")))
df_cust = df_cust.withColumn("LAST_NAME", initcap(col("LAST_NAME")))
df_cust = df_cust.withColumn("Full_STREET_ADDRESS", concat(col("APT_NO"),lit(','),col("STREET_NAME")))
df_cust=df_cust.withColumn("CUST_ZIP",col("CUST_ZIP").cast("int"))
df_cust=df_cust.withColumn("SSN",col("SSN").cast("int"))
df_cust=df_cust.withColumn("CUST_PHONE",col("CUST_PHONE").cast("string"))
df_cust=df_cust.withColumn("LAST_UPDATED",col("LAST_UPDATED").cast("timestamp"))
df_cust = df_cust.drop("APT_NO", "STREET_NAME")
df_cust.show(2)


+----------------+------------+-------------+-------------------+----------+----------+--------+----------+---------+-------------------+-----------+---------+--------------------+
|  CREDIT_CARD_NO|   CUST_CITY| CUST_COUNTRY|         CUST_EMAIL|CUST_PHONE|CUST_STATE|CUST_ZIP|FIRST_NAME|LAST_NAME|       LAST_UPDATED|MIDDLE_NAME|      SSN| Full_STREET_ADDRESS|
+----------------+------------+-------------+-------------------+----------+----------+--------+----------+---------+-------------------+-----------+---------+--------------------+
|4210653310061055|     Natchez|United States|AHooper@example.com|   1237818|        MS|   39120|      Alec|   Hooper|2018-04-21 11:49:02|         wm|123456100|656,Main Street N...|
|4210653310102868|Wethersfield|United States|EHolman@example.com|   1238933|        CT|    6109|      Etta|   Holman|2018-04-21 11:49:02|    brendan|123453023|   829,Redwood Drive|
+----------------+------------+-------------+-------------------+----------+----------+--------

In [6]:
df_cust.printSchema()

root
 |-- CREDIT_CARD_NO: string (nullable = true)
 |-- CUST_CITY: string (nullable = true)
 |-- CUST_COUNTRY: string (nullable = true)
 |-- CUST_EMAIL: string (nullable = true)
 |-- CUST_PHONE: string (nullable = true)
 |-- CUST_STATE: string (nullable = true)
 |-- CUST_ZIP: integer (nullable = true)
 |-- FIRST_NAME: string (nullable = true)
 |-- LAST_NAME: string (nullable = true)
 |-- LAST_UPDATED: timestamp (nullable = true)
 |-- MIDDLE_NAME: string (nullable = true)
 |-- SSN: integer (nullable = true)
 |-- Full_STREET_ADDRESS: string (nullable = true)



In [7]:
from pyspark.sql.functions import regexp_replace

df_cust = df_cust.withColumn('CUST_PHONE', regexp_replace('CUST_PHONE', r'(\d{3})(\d{4})', r'(000)$1-$2'))
df_cust.show(2)


+----------------+------------+-------------+-------------------+-------------+----------+--------+----------+---------+-------------------+-----------+---------+--------------------+
|  CREDIT_CARD_NO|   CUST_CITY| CUST_COUNTRY|         CUST_EMAIL|   CUST_PHONE|CUST_STATE|CUST_ZIP|FIRST_NAME|LAST_NAME|       LAST_UPDATED|MIDDLE_NAME|      SSN| Full_STREET_ADDRESS|
+----------------+------------+-------------+-------------------+-------------+----------+--------+----------+---------+-------------------+-----------+---------+--------------------+
|4210653310061055|     Natchez|United States|AHooper@example.com|(000)123-7818|        MS|   39120|      Alec|   Hooper|2018-04-21 11:49:02|         wm|123456100|656,Main Street N...|
|4210653310102868|Wethersfield|United States|EHolman@example.com|(000)123-8933|        CT|    6109|      Etta|   Holman|2018-04-21 11:49:02|    brendan|123453023|   829,Redwood Drive|
+----------------+------------+-------------+-------------------+-------------+-

In [8]:
df_cust1 = df_cust.select("SSN","FIRST_NAME","MIDDLE_NAME","LAST_NAME","CREDIT_CARD_NO","Full_STREET_ADDRESS","CUST_CITY","CUST_STATE","CUST_COUNTRY","CUST_ZIP","CUST_PHONE","CUST_EMAIL","LAST_UPDATED")
df_cust1.show(2)

+---------+----------+-----------+---------+----------------+--------------------+------------+----------+-------------+--------+-------------+-------------------+-------------------+
|      SSN|FIRST_NAME|MIDDLE_NAME|LAST_NAME|  CREDIT_CARD_NO| Full_STREET_ADDRESS|   CUST_CITY|CUST_STATE| CUST_COUNTRY|CUST_ZIP|   CUST_PHONE|         CUST_EMAIL|       LAST_UPDATED|
+---------+----------+-----------+---------+----------------+--------------------+------------+----------+-------------+--------+-------------+-------------------+-------------------+
|123456100|      Alec|         wm|   Hooper|4210653310061055|656,Main Street N...|     Natchez|        MS|United States|   39120|(000)123-7818|AHooper@example.com|2018-04-21 11:49:02|
|123453023|      Etta|    brendan|   Holman|4210653310102868|   829,Redwood Drive|Wethersfield|        CT|United States|    6109|(000)123-8933|EHolman@example.com|2018-04-21 11:49:02|
+---------+----------+-----------+---------+----------------+-------------------

In [19]:
df_cust1.write.format("jdbc")\
.mode("append")\
.option("driver","com.mysql.cj.jdbc.Driver")\
.option("user","")\
.option("url","jdbc:mysql://localhost:3306/creditcard_capstone")\
.option("dbtable","creditcard_capstone.CDW_SAPP_CUSTOMER ")\
.option("password","")\
.save()